# Beam Loading in the linear regime (beam driver)

Notebook developed by Lance Hildebrand using scripts made by Yujian Zhao

## Theory

In general, each slice of a beam in a wakefield will be accelerated at different rates due to the variation in the accelerating field, $E_z$. This will lead energy spread growth in the beam, which usually isn't desirable. The way to avoid this issue is called beam loading the wake.

In this notebook, we will focus on the linear regime, specifically a two-bunch scenario. Here the idea is quite simple. The total $E_z$ felt by the witness bunch will be the field created by the driver plus the one created by itself, via superposition. Therefore, for given drive bunch parameters, we can calulate the wake and therefore we can tailor a witness beam with a profile that will create a wake that will flatten the total field (first shown in Katsouleas et al., 1986). We know in the linear regime the wake behind the driver goes as $\cos\xi$. So we need a witness bunch with a wake that has an cosine part $\pi$ out of phase with the driver to cancel the $\xi$ dependence of the wake and a constant part equal to the value of the driver wake at the head of the witness bunch. It turns out a triangular shaped bunch can create such a wake. Gaussian beams can load a wake approximately but not perfectly. Now let's look at the math. For all of the following we will use normalized units.

The triangular bunch has four parameters we can tune: the max charge density, the transverse spot size, and the location of the head and the tail of the bunch. The profile is of the form (using $\xi=ct-z$)

$\displaystyle \rho=\rho_0\frac{\xi_f-\xi}{\xi_f-\xi_0}$.

We can calculate the on-axis field from this exactly (c.f. linear pwfa notebook),

$\displaystyle E_{z,w}=-R_w(0)\frac{\rho_0}{\xi_f-\xi_0} \int_{\xi_0}^\xi d\xi' (\xi_f-\xi') \cos (\xi-\xi')$,

where $R_w(0)$ is the radial function on-axis for the witness beam defined in the linear pwfa notebook. We have

$\displaystyle E_{z,w}=-R_w(0)\frac{\rho_0}{\xi_f-\xi_0}\left[ (\xi_f-\xi_0)\sin (\xi-\xi_0)+\cos (\xi-\xi_0)-1 \right]$.

We can use trig identites to separate $\sin\xi$ and $\cos\xi$ terms,

$\displaystyle E_{z,w}=-R_w(0)\frac{\rho_0}{\xi_f-\xi_0}\left[ \left(\sin\xi_0+(\xi_f-\xi_0)\cos\xi_0\right)\sin\xi+\left(\cos\xi_0-(\xi_f-\xi_0)\sin\xi_0\right)\cos\xi -1 \right]$.

Alright, now we want to choose parameters such that the $\sin\xi$ term goes away and the coefficient on the $\cos\xi$ is opposite of the driver amplitude, call it $E_0$ for now. The astute observer will notice we have one more constraint that the constant here needs to be $E_0\cos\xi_0$. We will see this is redundant with the second one. The constraints are

$\displaystyle (\xi_f-\xi_0)\cos\xi=-\sin\xi$

$\displaystyle \cos\xi-(\xi_f-\xi_0)\sin\xi_0=\frac{(\xi_f-\xi_0)E_0}{R_w(0)\rho_0}$,

or, simplified,

$\displaystyle \xi_f=\xi_0-\tan\xi_0$

$\displaystyle \frac{\rho_0 R_w(0)}{E_0}=-\sin\xi_0$.

We have one degree of freedom. So say for example we know the beam separation, then these constraints will tell us how long and how much charge we need to flatten the wake. 

An important thing to note is that if we are loading with an electron beam we need the slope of the wake to be negative at the head of the beam, i.e. $\pi/2<\xi_0<\pi$. If you are loading with positive charge you would need to put it in $3\pi/2<\xi_0<2\pi$.

For reference,

$\displaystyle E_0=\rho_{0}R(0)\sqrt{2\pi}\sigma_{z}e^{-\sigma_{z}^2/2}$

for a Gaussian bunch, and

$\displaystyle R(0)=\frac{1}{2}\sigma_r^2 e^{\sigma_r^2/2}\Gamma\left(0,\sigma_r^2/2\right)$

for a transverse Gaussian profile, where $\Gamma(n,x)=\int_x^\infty t^{n-1}e^{-t}dt$.

The next cell will allow you to explore beam loading with different drive and witness bunch parameters using these analytic formulas. The followings cells will allow you to use your predictions to set up and run a QuickPIC simulation to confirm the parameters you calculated load the wake.

In [1]:
from importlib import reload
import wakeplot

In [8]:
wakeplot = reload(wakeplot)

#Specify xi dimensions (shouldn't have to change this)
indz=8     #number of cells
box_z=[-3.0,10.0]     #min and max xi values

#Specify drive beam parameters (all in normalized units)
nb_driver=0.001   #beam denisty of driver normalized to the plasma density
z_center_driver=0.0     #location of the center of the drive beam profile
sigma_r_driver=0.5     #transv. spot size of the driver 
sigma_z_driver=0.4     #long. spot size of the driver

#Specify witness beam parameters
#### for positrons just specify nb_witness to be negative (note on the second plot the slider will still show as positive even though it's really negative)
nb_witness=0.062     #beam denisty of witness beam normalized to the plasma density (will result in error if density is out of the appropriate range)
z_front_witness=2.0    #location of the front of th\e witness beam (between pi/2 and pi)
z_back_witness=z_front_witness+0.8     #location of the back of the witness beam
sigma_r_witness=0.5   #transv. spot size of the witness beam

use_theory=True      #calculate the two free parameters from theory given the slider parameter. (if true the wake will always be flattened)
use_fixed_length=True     #for first plot (if use theory is false), this will force the beam to maintain the initially specified length

wakeplot.z1_interact(box_z,indz,nb_driver,z_center_driver,sigma_r_driver,sigma_z_driver,
                     nb_witness,sigma_r_witness,z_front_witness,z_back_witness,use_fixed_length,use_theory)

interactive(children=(FloatSlider(value=2.0, continuous_update=False, description='z1_w', max=3.1, min=1.6, st…

interactive(children=(SelectionSlider(continuous_update=False, description='nb_w', options=(('9.163e-05', 9.16…

## Simulation
### 1. Make an input file

In [ ]:
from importlib import reload
import qphelper
qphelper = reload(qphelper)
qphelper.makeWidgetsForInput()

### 2. Run QuickPIC simulation

In [ ]:
import quickpic
qphelper.deleteAllOutput()
quickpic.runqpic(rundir='rundir',inputfile='qpinput.json')

### 3. Visualize the output data

In [ ]:
qphelper = reload(qphelper)
qphelper.makeplot('rundir')